In [1]:
# 导入同花顺包和登录同花顺
import datetime as dt
import pandas as pd
from iFinDPy import *
THS_iFinDLogin("gdzq6888", "199212")

D:\Anaconda\Lib\site-packages\iFinDPy.pth


0

# 清仓天数

In [74]:
def liquidation_days(fund_code, date):
    #获取基金持股数据
    position_data = pd.DataFrame(THS_DP('fund_asset_allocation_stock_holding',fund_code+';'+date ,'fund_code:Y,fund_name:Y,report:Y,stock_code:Y,stock_name:Y,hold_num_stock:Y,hold_num_change_stock:Y,hold_num_ratio_float_stock:Y,hold_stock_amount:Y,net_value_ratio:Y,stock_mv_ratio:Y,investment_type:Y,management_company:Y').data)

    #获取持仓股票代码
    stock_code = str(list(position_data['stock_code'])).strip("[]")

    # 得到前X个月的日期字符串new_date
    format_date = datetime.strptime(date, "%Y-%m-%d")
    prev_30 = format_date-timedelta(days=30)
    prev_30 = prev_30.strftime("%Y-%m-%d")
    prev_90 = format_date-timedelta(days=90)
    prev_90 = prev_90.strftime("%Y-%m-%d")
    prev_180 = format_date-timedelta(days=180)
    prev_180 = prev_180.strftime("%Y-%m-%d")

    # 生成前30， 90， 180天的成交额进行对比
    deal_amount = pd.DataFrame(list(position_data['stock_code'])).rename(columns={0: "thscode"})
    for i in [prev_30,prev_90,prev_180]:
        a = THS_BD(stock_code,'ths_daily_avg_amt_int_stock',i +','+date).data
        a = a.rename(columns={"ths_daily_avg_amt_int_stock": i})
        deal_amount = pd.merge(deal_amount,a, on ="thscode", how="outer")

    # 取得三列中的最小值, 并与原数据集进行合并
    deal_amount['min_deal_amount'] = deal_amount.iloc[:,1:4].min(axis=1)
    deal_amount = deal_amount[['thscode', 'min_deal_amount']]
    deal_amount = deal_amount.rename(columns={"thscode": "stock_code"})
    merged_data = pd.merge(position_data, deal_amount, on='stock_code', how='left')

    # 得到日均成交限额
    merged_data['日均成交限额'] = merged_data['min_deal_amount'] * 0.01

    # 得到清仓天数和加权清仓天数
    merged_data['清仓天数'] = merged_data['hold_stock_amount'] * 10000 / merged_data['日均成交限额']
    merged_data['加权清仓天数'] = merged_data['清仓天数'] * merged_data['stock_mv_ratio'] / 100
    weighted_liquidation_days = merged_data['加权清仓天数'].sum()

    print("基金"+ fund_code + "的清仓天数为" + str(weighted_liquidation_days))+"天"

    return

In [75]:
liquidation_days('003567.OF', '2022-06-30')

基金003567.OF的清仓天数为152.1622635185788


# 换手率

In [42]:
def change_hand_rate(fund_code,date):
    # 得到基金的托管费率和托管费并计算报告期平均资产规模
    custodian_fee = THS_BD(fund_code,'ths_trustee_fee_fund;ths_mandate_fee_rate_fund',date +';').data
    formated_date = datetime.strptime(date, "%Y-%m-%d")
    ## 如果是半年报，则托管费乘以二，年报则不乘2
    if formated_date.month == 6:
        custodian_fee['average_asset_size'] = (custodian_fee['ths_trustee_fee_fund']*2)/(custodian_fee['ths_mandate_fee_rate_fund']/100)
    else:
        custodian_fee['average_asset_size'] = (custodian_fee['ths_trustee_fee_fund'])/(custodian_fee['ths_mandate_fee_rate_fund']/100)

    # 得到基金的股票买入和卖出总额, 并合并数据
    buy_sell = THS_BD(fund_code,'ths_total_buy_stock_cost_rp_fund;ths_total_sell_stock_income_rp_fund',date+';'+date).data
    merged_data = pd.merge(buy_sell,custodian_fee,on='thscode', how='outer')

    # 计算换手率
    change_hand_rate = (merged_data['ths_total_buy_stock_cost_rp_fund']+merged_data['ths_total_sell_stock_income_rp_fund'])/merged_data['average_asset_size']
    change_hand_rate[]

    print(change_hand_rate)

    return

In [43]:
change_hand_rate('003567.OF','2022-06-30')

0    1.276102
dtype: float64


In [37]:
change_hand_rate('003567.OF','2021-12-31')

0    7.803471
dtype: float64


In [38]:
change_hand_rate('003095.OF','2022-06-30')

0    0.202917
dtype: float64


In [39]:
change_hand_rate('003095.OF','2021-12-31')

0    1.637787
dtype: float64


In [44]:
change_hand_rate('000001.OF','2022-06-30')

0    1.590544
dtype: float64


# 超限比率

# 以下是草稿

In [16]:



# 得到基金的托管费率和托管费并计算报告期平均资产规模
custodian_fee = THS_BD('003567.OF','ths_trustee_fee_fund;ths_mandate_fee_rate_fund','2022-06-30;').data
custodian_fee['average_asset_size'] = (custodian_fee['ths_trustee_fee_fund']*2)/(custodian_fee['ths_mandate_fee_rate_fund']/100)

In [19]:
# 得到基金的股票买入和卖出总额
buy_sell = THS_BD('003567.OF','ths_total_buy_stock_cost_rp_fund;ths_total_sell_stock_income_rp_fund','2022-06-30;2022-06-30').data

In [22]:
merged_data = pd.merge(buy_sell,custodian_fee,on='thscode', how='outer')

In [26]:
change_hand_rate = (merged_data['ths_total_buy_stock_cost_rp_fund']+merged_data['ths_total_sell_stock_income_rp_fund'])/merged_data['average_asset_size']

In [27]:
change_hand_rate

0    1.276102
dtype: float64

In [18]:
THS_BD('003567.OF','ths_stock_invest_cost_fund;ths_stock_invest_fund','2022-06-30;2022-06-30').data

,thscode,ths_stock_invest_cost_fund,ths_stock_invest_fund
0,003567.OF,9.758806e+09,9.968603e+09
